In [2]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

In [3]:
class Optimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{'lr': 0}]

    def step(self): None
    def zero_grad(self, set_to_none: bool = False): None


class Scheduler:
    def step(self): None

# Model Architecture
Most competitive neural sequence transduction models have an encoder-decoder structure. Here, the encoder maps an input sequence of symbol reprensentations $(x_1, ..., x_n)$ to a sequence of continuous representations $Z = (z_1, ..., z_n)$. Given *z*, the decoder then generates an output sequence $(y_1, ..., y_n)$ of symbols one element at a time. At each step the model is auto-regressive, consuming the previously generated symbols as additional input when generating the next

In [5]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many other models.
    """
    def __init__(self, encoder: nn.Module, decoder: nn.Module, src_embed: torch.Tensor, tgt_embed: torch.Tensor, generator: callable):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src: torch.Tensor, tgt: torch.Tensor, src_mask: torch.Tensor, tgt_mask: torch.Tensor) -> torch.Tensor:
        "Take in and process masked src and target sequences"
        encoded = self.encode(src, src_mask)
        decoded = self.decode(encoded, src_mask, tgt, tgt_mask)
        return decodedß

    def encode(self, src: torch.Tensor, src_mask: torch.Tensor) -> torch.Tensor:
        embeddings = self.src_embed(src)
        return self.encoder(embeddings, src_mask)

    def decode(self, memory: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor) -> torch.Tensor:
        embeddings = self.tgt_embed(tgt)
        return self.decoder(embeddings, memory, src_mask, tgt_mask)